In [156]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains
import csv
import scrapy
import time
import re
import pandas as pd

In [373]:
#you will need to put your own path to the firefox binary here
#binary = FirefoxBinary()
legistar = 'https://milwaukee.legistar.com/Legislation.aspx'

def visit_site(legistar):
    mkeDriver = webdriver.Firefox(executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
    mkeDriver.get(legistar)
    #return mkeDriver


In [375]:
mkeDriver2 = visit_site(legistar)

In [385]:
test = get_next_page(mkeDriver2)

[]


In [481]:
def select_all_years(mkeDriver):
    years = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstYears_Input")
    years.send_keys('All Years')
    years.submit()
    #return years

In [581]:
#makes it so that you have max results

def get_max_results(mkeDriver):
    step1 = mkeDriver.find_elements_by_id("ctl00_ContentPlaceHolder1_menuMain")[0]
    action = ActionChains(mkeDriver)
    action.click(step1.find_elements_by_class_name("rmItem ")[1])
    #action.click(step1.find_elements_by_class_name("rmItem ")[1].find_element_by_css_selector("a"))
    action.perform()
    time.sleep(1)
    step2 = step1.find_elements_by_class_name("rmItem ")[1].parent
    #print(step2.text)
    action = ActionChains(mkeDriver)
    # action_test2 = ActionChains(testDriver)
    action.move_to_element(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.click(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.perform()
    #return mkeDriver

In [332]:
# re.search(r'(?<=of\s)[0-9]+',mkeDriver.find_elements_by_class_name("rgPagerCell")[0]\
#             .find_element_by_class_name("rgInfoPart").text)[0]

In [205]:
#need to consider the recursion involved in looping through pages OR use above

In [589]:
#locats page link 
def get_next_page(mkeDriver):
    pgs_drop = mkeDriver.find_elements_by_class_name("rgPagerCell")
    #print(pgs_drop)
    try:
        return pgs_drop[0].find_element_by_xpath("//a[@class='rgCurrentPage']/following-sibling::a")
    except:
        return None

def go_to_next_page(next_page):
    next_page.click()

In [510]:
def select_ordinance(mkeDriver):
    type_ordinance = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_Arrow").click()
    legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
                .find_elements_by_class_name("rcbItem")
    print(legislation_type[14].text)
    actions = ActionChains(mkeDriver)
    actions.move_to_element(legislation_type[14])
    actions.click(legislation_type[14])
    actions.perform()
    #choose_ordinance.perform()#.double_click()
#type_legislation.s
#type_legislation.submit()

In [444]:
# legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
#                 .find_elements_by_class_name("rcbItem")
# hover_ord = ActionChains(mkeDriver).move_to_element(legislation_type[14])
# hover_ord.perform().send_keys("Ordinance").double_click()

In [483]:
def start_search(mkeDriver):
    s_button = mkeDriver.find_element_by_id("visibleSearchButton")
    s_button.click()
    #return mkeDriver

In [335]:
def get_records_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridMain_ctl00")
    r_table_body = records_table.find_element_by_css_selector("tbody")
    return r_table_body

In [212]:
# for i in records_table.find_elements_by_css_selector('a'):
#     if 'https' in i.get_attribute('href'):
#         print(i.get_attribute('href'))

In [336]:
#r_table_body = records_table.find_element_by_css_selector("tbody")

In [ ]:
#can change # of records per page :)

In [590]:
#if status passed
import time

def get_meta_data(legistar):
    count=0
    leg_dict = {}
    #CALL OTHER REQUISITE FUNCTIONS FIRST
    mkeDriver = visit_site(legistar)
    #time.sleep(5)
    select_all_years(mkeDriver)
    time.sleep(5)
    select_ordinance(mkeDriver)
    start_search(mkeDriver)
    time.sleep(5)
    get_max_results(mkeDriver)

    
    while True:
        time.sleep(3)
        r_table_body = get_records_table(mkeDriver)
        for row in r_table_body.find_elements_by_css_selector('tr'):
            file_num = row.find_element_by_css_selector('a').text
            leg_dict[file_num] = {}
            leg_dict[file_num]['file_link'] = row.find_element_by_css_selector('a').get_attribute('href')
            for d in row.find_elements_by_css_selector('td'):
                #if leg_dict[file_num]['status'] in ('Passed',"Mayor's Office",'Dead','Placed On File'): use this later for going through links
                leg_dict[file_num]['type'] = d.text
                leg_dict[file_num]['status'] = d.text
                leg_dict[file_num]['created_date'] = d.text
                leg_dict[file_num]['final_action'] = d.text
                leg_dict[file_num]['title'] = d.text
        cur_page = mkeDriver.find_elements_by_class_name("rgPagerCell")[0]\
                .find_element_by_class_name("rgCurrentPage")

        
        count+=1
        
        next_page = get_next_page(mkeDriver)
        
        if not next_page:
            return 
        else:
            next_page.click()
    

In [591]:
#leg_dict

In [ ]:
meta_legis_dict = get_meta_data(legistar)

Ordinance
